In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [ ]:
path = "/kaggle/input/competitive-data-science-predict-future-sales/"

items = pd.read_csv(path+'/items.csv')
item_cats = pd.read_csv(path+'/item_categories.csv')
shops = pd.read_csv(path+'/shops.csv')
sales = pd.read_csv(path+'/sales_train.csv')
test = pd.read_csv(path+'/test.csv')
submission = pd.read_csv(path+'/sample_submission.csv')

In [ ]:
pre_df = sales.copy()
pre_df = pre_df.pivot_table(
    index=['shop_id', 'item_id'],
    values=['item_cnt_day'],
    columns=['date_block_num'],
    fill_value=0,
    aggfunc='sum'
).reset_index()

pre_df

In [ ]:
full_train_df = test.copy()
full_train_df = full_train_df.merge(pre_df, how='left', on=['shop_id', 'item_id']).fillna(0).drop(
    ['ID', 'shop_id', 'item_id'], axis=1)
full_train_df

In [ ]:
X_train, y_train = full_train_df.values[:,:-2], full_train_df.values[:, -2:-1].ravel()
X_valid, y_valid = full_train_df.values[:,1:-1], full_train_df.values[:, -1:].ravel()
X_test = full_train_df.values[:, 2:]

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=3)

# **MLP**

In [ ]:
modelMLP = Sequential()
modelMLP.add(Dense(8, activation='relu', input_dim=X_train.shape[1]))
modelMLP.add(Dense(1))
modelMLP.compile(optimizer='adam', loss='mse')
history = modelMLP.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))

In [ ]:
MLPpred = modelMLP.predict(X_valid, verbose=0)
MLPpred = MLPpred.reshape((MLPpred.shape[0]))
MLPresults = metrics(MLPpred, y_valid)
MLPresults

In [ ]:
MLPnovember = modelMLP.predict(X_test, verbose=0)
MLPnovember = MLPnovember.reshape((MLPnovember.shape[0]))

# **CNN**

In [ ]:
X_train = X_train.reshape((X_test.shape[0], X_train.shape[1], 1))
X_valid = X_valid.reshape((X_test.shape[0], X_valid.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
modelCNN = Sequential()
modelCNN.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
modelCNN.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
modelCNN.add(Conv1D(filters=16, kernel_size=2, activation='relu'))
modelCNN.add(Conv1D(filters=8, kernel_size=2, activation='relu'))
modelCNN.add(MaxPooling1D(pool_size=2))
modelCNN.add(Flatten())
modelCNN.add(Dense(1))
modelCNN.compile(optimizer='adam', loss='mse')
history = modelCNN.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))

In [ ]:
CNNpred = modelCNN.predict(X_valid, verbose=0)
CNNpred = CNNpred.reshape((CNNpred.shape[0]))
CNNresults = metrics(CNNpred, y_valid)
CNNresults

In [ ]:
CNNnovember = modelCNN.predict(X_test, verbose=0)
CNNnovember = CNNnovember.reshape((CNNnovember.shape[0]))

# **LSTM**

In [ ]:
modelLSTM = Sequential()
modelLSTM.add(LSTM(16, activation='relu', input_shape=(X_train.shape[1], 1)))
modelLSTM.add(Dense(1))
modelLSTM.compile(optimizer='adam', loss='mse')
history = modelLSTM.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))

In [ ]:
LSTMpred = modelLSTM.predict(X_valid, verbose=0)
LSTMpred = LSTMpred.reshape((LSTMpred.shape[0]))
LSTMresults = metrics(LSTMpred, y_valid)
LSTMresults

In [ ]:
LSTMnovember = modelLSTM.predict(X_test, verbose=0)
LSTMnovember = LSTMnovember.reshape((LSTMnovember.shape[0]))

# **CNN-LSTM**

In [ ]:
subsequences = 2
timesteps = X_train.shape[1]//subsequences
X_train = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))
X_valid = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))
X_test = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))

In [ ]:
modelCNNLSTM = Sequential()
modelCNNLSTM.add(TimeDistributed(Conv1D(filters=64, kernel_size=8, activation='relu'), input_shape=(None, X_train.shape[2], X_train.shape[3])))
modelCNNLSTM.add(TimeDistributed(MaxPooling1D(pool_size=4)))
modelCNNLSTM.add(TimeDistributed(Flatten()))
modelCNNLSTM.add(LSTM(64, activation='relu'))
modelCNNLSTM.add(Dense(1))
modelCNNLSTM.compile(optimizer='adam', loss='mse')
history = modelCNNLSTM.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))

In [ ]:
CNNLSTMpred = modelCNNLSTM.predict(X_valid, verbose=0)
CNNLSTMpred = CNNLSTMpred.reshape((CNNLSTMpred.shape[0]))
CNNLSTMresults = metrics(CNNLSTMpred, y_valid)
CNNLSTMresults

In [ ]:
CNNLSTMnovember = modelCNNLSTM.predict(X_test, verbose=0)
CNNLSTMnovember = CNNLSTMnovember.reshape((CNNLSTMnovember.shape[0]))

# **ConvLSTM**

In [ ]:
X_train = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_valid = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_test = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))

In [ ]:
modelConvLSTM = Sequential()
modelConvLSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='relu', return_sequences=True,input_shape=(X_train.shape[1], 1, X_train.shape[3], X_train.shape[4])))
modelConvLSTM.add(ConvLSTM2D(filters=16, kernel_size=(1,2), activation='relu'))
modelConvLSTM.add(Flatten())
modelConvLSTM.add(Dense(1))
modelConvLSTM.compile(optimizer='adam', loss='mse')
history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))

In [ ]:
ConvLSTMpred = modelConvLSTM.predict(X_valid, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
ConvLSTMresults = metrics(ConvLSTMpred, y_valid)
ConvLSTMresults

In [ ]:
ConvLSTMnovember = modelConvLSTM.predict(X_test, verbose=0)
ConvLSTMnovember = ConvLSTMnovember.reshape((ConvLSTMnovember.shape[0]))

# **Results**

In [ ]:
names = ['MLP', 'CNN', 'LSTM', 'CNN-LSTM', 'ConvLSTM']
pd.DataFrame([MLPresults, 
              CNNresults,
              LSTMresults,
              CNNLSTMresults,
              ConvLSTMresults],
              index = names)

# **Predict November**

In [ ]:
results = {
    'MLP':MLPnovember,
    'CNN':CNNnovember,
    'LSTM':LSTMnovember,
    'CNNLSTM':CNNLSTMnovember,
    'ConvLSTM':ConvLSTMnovember
}

In [ ]:
resultdf = pd.DataFrame.from_dict(results)
resultdf = pd.concat([test, resultdf], axis = 1).drop(['ID', 'shop_id', 'item_id'],axis = 1)
resultdf

In [ ]:
meanresult = resultdf.mean(axis=1).rename("item_cnt_month").reset_index().rename(columns={'index': 'ID'})
meanresult

# **Save Models & Results**

In [ ]:
meanresult.to_csv('./output.csv', index=False)

In [ ]:
modelMLP.save('./modelMLP.h5')
modelCNN.save('./modelCNN.h5')
modelLSTM.save('./modelLSTM.h5')
modelCNNLSTM.save('./modelCNNLSTM.h5')
modelConvLSTM.save('./modelConvLSTM.h5')